In [1]:
import copy
import subprocess
from pathlib import Path

from tqdm import tqdm

from common.constants import ACTION_NAMES_IN_TEST, ALL_ACTION_NAMES, ALL_SELF_ACTIONS
from config_utils import base_model_from_file, base_model_to_file, base_model_to_str
from gbdt.configs import GBDT_TrainConfig, LightGBMConfig
from gbdt.helpers import get_train_config, is_fully_trained, read_metrics
from gbdt.ready_configs import make_default_data_split_config
from gbdt.rebalance_utils import (
    DownsampleParams,
    SampleCoefsParams,
    TestDownsampleParams,
    get_labs_in_test_with_action,
)
from optuna_utils import (
    apply_params_lightgbm,
    run_optuna_multirun,
    suggest_params_lightgbm,
)

def run_training(base_config: GBDT_TrainConfig) -> bool:
    config = copy.deepcopy(base_config)
    name = config.group
    cv = config.data_split_config.test_fold

    config.run_name = f"{name}_cv{cv}"
    save_dir = Path("train_logs") / name / f"cv{cv}"

    if is_fully_trained(name, cv):
        print(
            f"Final model for {config.run_name} already exists. Do NOT run training..."
        )
        return False
    else:
        print(f"Running training for '{config.run_name}'...")

    config.save_dir = str(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    config_path = save_dir / "train_config.json"

    base_model_to_file(config, config_path)
    subprocess.run(["python", "-m", "gbdt.train", "--config", config_path], check=False)
    return True

In [2]:
from gbdt.helpers import GBDT_Model_Type, get_any_train_config, model_type_by_name
from postprocess.postprocess_utils import top_models_by_oof_per_action, top_models_by_oof_per_action_per_lab


types = [t for t in GBDT_Model_Type]
models_dict_per_type = {}
for t in types:
    models_dict_per_type[t] = top_models_by_oof_per_action_per_lab(
        take_filter=lambda name: model_type_by_name(name) == t
    )

for (action, lab), model in tqdm(models_dict_per_type[GBDT_Model_Type.xgboost].items()):
    if action != "attack":
        continue
    name = model.name
    config = get_any_train_config(name)
    if not config.use_wandb:
        continue
    print(f"name = {name}, lab={lab}")
    config.features_config.hflip = True
    config.group += "-hflip"
    config.train_downsample_params.total_duration_cap /= 2
    for f in range(1):
        config.data_split_config.test_fold = f
        run_training(config)
        raise ValueError

  0%|          | 0/89 [00:00<?, ?it/s]

name = xgboost-attack-baseline-1-passive-0.7-NiftyGoldfinch-x64, lab=AdaptableSnail
Running training for 'xgboost-attack-baseline-1-passive-0.7-NiftyGoldfinch-x64-hflip_cv0'...


calc_features[attack]: 100%|███████████████████| 28/28 [00:11<00:00,  2.46it/s]


X_train dims: (1003640, 1020)
X_test dims: (1009098, 1020)

*** Stats for train begin***
AdaptableSnail       (  7.22%|w=  1.30%) [72480]: passive (72138,  99.53%|w= 70.00%), attack  (  342,   0.47%|w= 30.00%)
CRIM13               (  7.12%|w=  1.30%) [71474]: passive (69386,  97.08%|w= 70.00%), attack  ( 2088,   2.92%|w= 30.00%)
CalMS21_supplemental (  7.18%|w=  1.30%) [72022]: passive (63730,  88.49%|w= 70.00%), attack  ( 8292,  11.51%|w= 30.00%)
CalMS21_task1        (  7.13%|w=  1.30%) [71516]: passive (62504,  87.40%|w= 70.00%), attack  ( 9012,  12.60%|w= 30.00%)
CalMS21_task2        (  7.12%|w=  1.30%) [71458]: passive (68336,  95.63%|w= 70.00%), attack  ( 3122,   4.37%|w= 30.00%)
DeliriousFly         (  7.15%|w=  1.30%) [71732]: passive (70010,  97.60%|w= 70.00%), attack  ( 1722,   2.40%|w= 30.00%)
ElegantMink          (  7.12%|w=  1.30%) [71448]: passive (67808,  94.91%|w= 70.00%), attack  ( 3640,   5.09%|w= 30.00%)
InvincibleJellyfish  (  7.08%|w=  1.30%) [71088]: passive (68896

wandb: Currently logged in as: pavel7x7 (pavel7x7x7) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /home/pavel/wandb/wandb/run-20251124_163937-d0u0krxd
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run xgboost-attack-baseline-1-passive-0.7-NiftyGoldfinch-x64-hflip_cv0
wandb: ⭐️ View project at https://wandb.ai/pavel7x7x7/social-action-mice
wandb: 🚀 View run at https://wandb.ai/pavel7x7x7/social-action-mice/runs/d0u0krxd
XGB Training:  30%|███████▏                | 1500/5000 [01:28<03:26, 16.92it/s]
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                        best_prod-avg-pr-auc ▁
wandb:                   best_step_prod-avg-pr-auc ▁
wandb:             test-per-action-log-loss/attack ███▇▇▆▆▅▅▄▄▄▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               test-per-action-pr-auc/attack ▁▃▄▄▅▅▆▆▆

xgboost-attack-baseline-1-passive-0.7-NiftyGoldfinch-x64-hflip/cv0
Writing y_true for cv=cv0: (1009098, 1)
Predict step= 200, wall time: 0.99291
    time per 1M samples and 300 trees: 1.47594
Predict step= 300, wall time: 1.20056
    time per 1M samples and 300 trees: 1.18974
Predict step= 400, wall time: 1.49414
    time per 1M samples and 300 trees: 1.11050
Predict step= 500, wall time: 1.63187
    time per 1M samples and 300 trees: 0.97030
Predict step= 850, wall time: 1.78841
    time per 1M samples and 300 trees: 0.62551
Predict step= 900, wall time: 1.77328
    time per 1M samples and 300 trees: 0.58576
Predict step= 950, wall time: 1.82382
    time per 1M samples and 300 trees: 0.57075
Predict step=1000, wall time: 1.83019
    time per 1M samples and 300 trees: 0.54411
Predict step=1100, wall time: 1.83350
    time per 1M samples and 300 trees: 0.49554
Predict step=1200, wall time: 1.79345
    time per 1M samples and 300 trees: 0.44432
Predict step=1300, wall time: 1.81561
    t

  7%|▋         | 6/89 [04:04<56:21, 40.74s/it]


ValueError: 